# Wikipedia Notable Life Expectancies

# [Notebook 4 of : Data Cleaning Part 3](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean3_thanak_2022_06_23.ipynb)

## Context

The


## Objective

The

### Data Dictionary

- Feature: Description

## Importing Necessary Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean2.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean2", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 132652 rows and 21 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,British dancer,ballet designer and director,None,None,None,None,None,None,None,None,86.0,None
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,Irish economist,writer,and academic,None,None,None,None,None,None,None,68.0,None


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death
132650,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2,2022,June,(1980),None,Russian volleyball player,Olympic champion and coach,None,None,None,None,None,None,None,None,69.0,None
132651,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,Chinese engineer,member of the Chinese Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death
120692,24,Anna Koźmińska,", 101, Polish recipient of Righteous Among the Nations award.",https://en.wikipedia.org/wiki/Anna_Ko%C5%BAmi%C5%84ska,5,2021,March,None,None,Polish recipient of Righteous Among the Nations award,None,None,None,None,None,None,None,None,None,101.0,None
116398,13,Jim Pace,", 59, American racing driver, complications of COVID-19.",https://en.wikipedia.org/wiki/Jim_Pace_(racing_driver),2,2020,November,None,None,American racing driver,complications of COVID-19,None,None,None,None,None,None,None,None,59.0,None
77797,1,Božo Bakota,", 64, Croatian footballer .",https://en.wikipedia.org/wiki/Bo%C5%BEo_Bakota,7,2015,October,"(NK Zagreb, SK Sturm Graz)",None,Croatian footballer,None,None,None,None,None,None,None,None,None,64.0,None
39717,5,Dave Pureifory,", 59, American football player , prostate cancer.",https://en.wikipedia.org/wiki/Dave_Pureifory,6,2009,March,(Detroit Lions),None,American football player,prostate cancer,None,None,None,None,None,None,None,None,59.0,None
43502,30,Ruth Cohn,", 97, German psychotherapist.",https://en.wikipedia.org/wiki/Ruth_Cohn,6,2010,January,None,None,German psychotherapist,None,None,None,None,None,None,None,None,None,97.0,None


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132652 entries, 0 to 132651
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   day             132652 non-null  object 
 1   name            132652 non-null  object 
 2   info            132652 non-null  object 
 3   link            132652 non-null  object 
 4   num_references  132652 non-null  object 
 5   year            132652 non-null  int64  
 6   month           132652 non-null  object 
 7   info_parenth    49830 non-null   object 
 8   info_1          35 non-null      object 
 9   info_2          132604 non-null  object 
 10  info_3          62571 non-null   object 
 11  info_4          12605 non-null   object 
 12  info_5          1497 non-null    object 
 13  info_6          216 non-null     object 
 14  info_7          31 non-null      object 
 15  info_8          6 non-null       object 
 16  info_9          1 non-null       object 
 17  info_10   

<IPython.core.display.Javascript object>

#### Loading `nation_country_dict` from Pickle File to Dictionary `nation_map`

In [6]:
# Load the nation_country_dict
with open("nation_country_dict.pkl", "rb") as f:
    nation_map = pickle.load(f)

<IPython.core.display.Javascript object>

## Extracting Nationality Continued
Here is the approach we will take:
- The plan will be to save the country name, in lieu of nationality, in new `place_1` and `place_2` columns as it is standardized for the various associated nationality values.
- First, we will update the keys and values in `nation_map` by replacing hyphens with a single space.
- Then we will remove "-born" from the column we are searching, as well as replace "-" and "/" each with single spaces.  In this step, we can also remove leading and trailing periods and whitespace.
- We will proceed to search the numbered `info` columns in order checking as follows:
    1. if column value starts with a value in the dictionary:
        - save country to `place_1` and remove value from searched column.
    2. if `place_1` value has been found:
        - if updated column value starts with a value in the dictionary:
            - save country to `place_2` and remove value from searched column.
    3. Repeat steps 1 and 2 but comparing with country (dictionary keys)
    4. Check unique values for column starting with capital letters.
- It is tempting to shorten the process by simply searching for nationality and country values within the column value, but as there are entries containing first and second nationality, we have to proceed from left to right to extract these values optimally.

#### Removing "-" and "." from `nation_map`

In [7]:
# Removing hyphens from nation_map
nation_map = {
    key.replace("-", ""): value.replace("-", " ") for key, value in nation_map.items()
}

# Removing periods from nation_map
nation_map = {
    key.replace(".", ""): value.replace(".", " ") for key, value in nation_map.items()
}

<IPython.core.display.Javascript object>

#### Removing or Replacing Extra Characters in Numbered `info` Columns

In [8]:
%%time

# List of columns to treat
cols_lst = [
    "info_1",
    "info_2",
    "info_3",
    "info_4",
    "info_5",
    "info_6",
    "info_7",
    "info_8",
    "info_9",
    "info_10",
    "info_11",
    'info_parenth'
]

# Dictionary of keys to find and values to replace keys
replace_dict = {'-born': '', '–born': '', '-': ' ', '–': ' ', '/': ' ', '.': ' ', '(': '', ')': ''}

# For loop to find and replace characters in replace_dict in columns in cols_list
# and strip any leading or trailing periods or whitespace
for column in cols_lst:
    for key, value in replace_dict.items():
        for index in df[column].notna().index:
            item = df.loc[index, column]
            if item:
                df.loc[index, column] = item.replace(key, value).strip()
                
# Chime notification when cell successfully executes
chime.success()

CPU times: total: 2min 36s
Wall time: 2min 36s


<IPython.core.display.Javascript object>

#### Checking `info_1` for `place_1`

In [9]:
# Column to check
column = "info_1"

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if item.startswith(nationality) or item.startswith(country):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column]
                .replace(nationality, "")
                .strip()
                .replace(country, "")
                .strip()
            )

# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1
32981,28,David Turnbull,". 92, American materials scientist.",https://en.wikipedia.org/wiki/David_Turnbull_(materials_scientist),8,2007,April,None,materials scientist,None,None,None,None,None,None,None,None,None,None,92.0,None,United States of America
19580,20,Dame Miraka Szászy,", 80. New Zealand Maori leader.",https://en.wikipedia.org/wiki/Mira_Sz%C3%A1szy,21,2001,December,None,Maori leader,None,None,None,None,None,None,None,None,None,None,80.0,None,New Zealand


<IPython.core.display.Javascript object>

#### Observations:
- `info_1` provides us a small sample on which to test code.
- We successfully extracted those `place_1` values, now we will do the same on the treated rows for `place_2`.

#### Checking `info_1` for `place_2`

In [10]:
# Column to check
column = "info_1"

# Extract to column
extract_to = "place_2"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df["place_1"].notna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if item.startswith(nationality) or item.startswith(country):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column]
                .replace(nationality, "")
                .strip()
                .replace(country, "")
                .strip()
            )

# Check a sample of rows
df.sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1
71126,4,Silver Deputy,", 29, Canadian Thoroughbred racehorse, euthanized.",https://en.wikipedia.org/wiki/Silver_Deputy,4,2014,October,None,None,Canadian Thoroughbred racehorse,euthanized,None,None,None,None,None,None,None,None,29.0,None,NaN
48688,8,Jim Keane,", 87, American football player .",https://en.wikipedia.org/wiki/Jim_Keane,0,2011,March,"Chicago Bears, Green Bay Packers",None,American football player,None,None,None,None,None,None,None,None,None,87.0,None,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Here we can see that the new column place_2 has not yet been added as there were not any matching values.
- Let us confirm by checking the remaining unique values in info_1.

#### Checking Remaining Unique Values in `info_1`

In [11]:
# Checking unique values
df["info_1"].unique()

array([None, 'politician', 'Olympic sprinter', 'gridiron football player',
       'writer', 'businessman', 'social psychologist', 'King of Nepal',
       'Maori leader', 'artist', 'English sports journalist',
       'Jules Engel', 'early', 'aka', 'Jr', 'professional wrestler',
       'automotive engineer', 'materials scientist', 'weightlifter',
       'common chimpanzee', '', 'Olympic athlete', 'actor',
       'Olympic gymnast', 'broadcaster and writer', 'Olympic swimmer',
       'Olympic boxer', 'Olympic wrestler', 'Olympic sailor',
       'basketball player', 'college basketball coach',
       'choral conductor', 'Tree of the Year'], dtype=object)

<IPython.core.display.Javascript object>

#### Obsservations:
- Neither "English" nor "Maori" are keys in the current dictionary.
- Maori is an ethnicity within the country of New Zealand, so for now, we will add it as a key our dictionary with the country value of New Zealand.  If we have matching first and second countries, we can later remove the first value.
- We will also add the key "English" with the country value 'United Kingdom of Great Britain and Northern Ireland'.
- Then, we can rerun the above code for `place_1` and `place_2`.
- The country value of "Nepal" is also present. We will hold off on extracting country names until we have first exhausted matching nationalities, as the Wikipedia field called for nationalities.

#### Updating `nation_map`

In [12]:
# Adding key: country pairs to nation_map
nation_map["English"] = nation_map["British"]
nation_map["Maori"] = nation_map["New Zealand"]

<IPython.core.display.Javascript object>

#### Re-checking `info_1` for `place_1`

In [13]:
# Column to check
column = "info_1"

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if item.startswith(nationality) or item.startswith(country):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column]
                .replace(nationality, "")
                .strip()
                .replace(country, "")
                .strip()
            )

# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1
99880,13,André Queillé,", 87 French Olympic boxer .",https://en.wikipedia.org/wiki/Andr%C3%A9_Queill%C3%A9,2,2018,December,1952,Olympic boxer,None,None,None,None,None,None,None,None,None,None,87.0,None,France
11825,23,Manuel Mejía Vallejo,", 75 Colombian writer.",https://en.wikipedia.org/wiki/Manuel_Mej%C3%ADa_Vallejo,2,1998,July,None,writer,None,None,None,None,None,None,None,None,None,None,75.0,None,Colombia


<IPython.core.display.Javascript object>

#### Re-checking `info_1` for `place_2`

In [14]:
# Column to check
column = "info_1"

# Extract to column
extract_to = "place_2"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df["place_1"].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if item.startswith(nationality) or item.startswith(country):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column]
                .replace(nationality, "")
                .strip()
                .replace(country, "")
                .strip()
            )

# Checking rows
df[df["place_2"].notna()]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
19580,20,Dame Miraka Szászy,", 80. New Zealand Maori leader.",https://en.wikipedia.org/wiki/Mira_Sz%C3%A1szy,21,2001,December,None,leader,None,None,None,None,None,None,None,None,None,None,80.0,None,New Zealand,New Zealand


<IPython.core.display.Javascript object>

#### Observations:
- Our code appears to be finding the matching values and assigning the corresponding country to the correct nation column.
- We see "New Zealand" added to both nation columns here, which was expected as both New Zealand and Maori are in the description
- As an aside, we will need to check our final values where `place_1` is "American" and `place_2` is "Indian" as our code will indicate United States and India, which may or may not be correct. 
- Now we can proceed to doing the same extraction on `info_2`.

#### Checking `info_2` for `place_1`

In [15]:
%%time

# Column to check
column = "info_2"

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if item.startswith(nationality) or item.startswith(country):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
            )

# Chime notification when cell successfully executes
chime.success()

CPU times: total: 2min 38s
Wall time: 2min 38s


<IPython.core.display.Javascript object>

In [16]:
# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
13086,20,Gene Siskel,", 53, American film critic and television journalist, brain cancer.",https://en.wikipedia.org/wiki/Gene_Siskel,61,1999,February,None,None,film critic and television journalist,brain cancer,None,None,None,None,None,None,None,None,53.0,None,United States of America,NaN
12330,27,Stirling Dickinson,", 89, American artist, car accident.",https://en.wikipedia.org/wiki/Stirling_Dickinson,17,1998,October,None,None,artist,car accident,None,None,None,None,None,None,None,None,89.0,None,United States of America,NaN


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for place_1

In [17]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')

There are 2275 remaining missing values for place_1.



<IPython.core.display.Javascript object>

#### Observations:
- We have captured the `place_1` value for the vast majority of entries.
- Next we will check for other variations on nationality usage.

#### Examining Unique Values of First Word in `info_1` if Upper Case

In [18]:
# Column to check
column = "info_2"

# Dataframe to check
dataframe = df[(df["place_1"].isna()) & (df[column].notna())]

# Checking set of first words in info_2 where place_1 is missing
first_words = set([item.split()[0] for item in dataframe[column] if item[0].isupper()])
print(f"There are {len(first_words)} unique values for first word in info_1.\n")
first_words

There are 302 unique values for first word in info_1.



{'AIDS',
 'ANC',
 'Abkhaz',
 'Abkhazian',
 'Aboriginal',
 'Actress',
 'African',
 'Afrikaans',
 'Afrikaner',
 'Afro',
 'Air',
 'Alfa',
 'All',
 'Alyawarre',
 'Amateur',
 'America',
 "America's",
 'Amrican',
 'Anglican',
 'Anglo',
 'Anguillan',
 'Arabic',
 'Archbishop',
 'Archdeacon',
 'Argentinian',
 'Aruba',
 'Aruban',
 'Assamese',
 'Associate',
 'Assyrian',
 'Athletics',
 'Aussie',
 'Austro',
 'Avarian',
 'Azorean',
 'BBC',
 'Baltic',
 'Basque',
 'Bavarian',
 'Benedictine',
 'Bermudan',
 'Bermudian',
 'Bessarabian',
 'Bletchley',
 'Bodo',
 'Bosnia',
 'Breton',
 'Brigadier',
 "Britain's",
 'Britsih',
 'California',
 'Californian',
 'Calypso',
 'Cantonese',
 'Caribbean',
 'Catalan',
 'Catholic',
 'Caymanian',
 'Ceylon',
 'Ceylonese',
 'Chagossian',
 'Chairman',
 'Chechen',
 'Cherokee',
 'Chief',
 'Chilian',
 'China',
 'Chiricahua',
 'Chuvash',
 'Circassian',
 'Civil',
 'Columbian',
 'Commandant',
 'Composer',
 'Computer',
 'Congo',
 'Congoleze',
 'Congresswoman;',
 'Cook',
 'Cornish',


<IPython.core.display.Javascript object>

#### Observations:
- We can see there are some remaining variations on how nationality was entered that are not yet in `nation_country_dict`.
- Let us add those now, then do another iteration searching `info_2`.
- Descriptions will be assigned to their geographical physical region where sovereign state is remote or nationality description is broad.

#### Hard-coding Additional Variations on Nationality

In [19]:
# Hard-coding remaining unique nationality/location descriptors
nation_map["ANC"] = nation_map["South African"]
nation_map["Abkhaz"] = nation_map["Georgian"]
nation_map["Abkhazian"] = nation_map["Georgian"]
nation_map["Aboriginal"] = nation_map["Australian"]
nation_map["African"] = "Africa"
nation_map["Afrikaans"] = nation_map["African"]
nation_map["Afrikaner"] = nation_map["African"]
nation_map["Afro"] = "Africa"
nation_map["Alyawarre"] = nation_map["Australian"]
nation_map["America"] = nation_map["US"]
nation_map["America's"] = nation_map["US"]
nation_map["Amrican"] = nation_map["US"]
nation_map["Anglo"] = "Europe"
nation_map["Anguillan"] = "Central America and the Caribbean"
nation_map["Antigua"] = nation_map["Antiguan"]
nation_map["Arabic"] = "Arab states"
nation_map["Argentinian"] = nation_map["Argentine"]
nation_map["Aruba"] = "Central America and the Caribbean"
nation_map["Aruban"] = nation_map["Aruba"]
nation_map["Assamese"] = nation_map["Indian"]
nation_map["Assyrian"] = "Middle East"
nation_map["Aussie"] = nation_map["Australian"]
nation_map["Australia"] = nation_map["Australian"]
nation_map["Australia's"] = nation_map["Australian"]
nation_map["Austria"] = nation_map["Austrian"]
nation_map["Austro"] = nation_map["Austrian"]
nation_map["Avarian"] = nation_map["Russian"]
nation_map["Azerbaijan"] = nation_map["Azerbaijani"]
nation_map["Azorean"] = nation_map["Portuguese"]
nation_map["Baltic"] = "Europe"
nation_map["Bangladesh"] = nation_map["Bangladeshi"]
nation_map["Barbados"] = "Central America and the Caribbean"
nation_map["Basque"] = "Europe"
nation_map["Bavarian"] = nation_map["German"]
nation_map["Belarus"] = nation_map["Belarusian"]
nation_map["Belarussian"] = nation_map["Belarusian"]
nation_map["Belgium"] = nation_map["Belgian"]
nation_map["Bermudan"] = "Central America and the Caribbean"
nation_map["Bermudian"] = "Central America and the Caribbean"
nation_map["Bessarabian"] = "Europe"
nation_map["Bletchley"] = nation_map["British"]
nation_map["Bodo"] = nation_map["Norwegian"]
nation_map["Bosnia"] = nation_map["Bosnian"]
nation_map["Breton"] = nation_map["French"]
nation_map["Britain's"] = nation_map["British"]
nation_map["Britsih"] = nation_map["British"]
nation_map["California"] = nation_map["US"]
nation_map["Californian"] = nation_map["US"]
nation_map["Cantonese"] = nation_map["Chinese"]
nation_map["Caribbean"] = "Central America and the Caribbean"
nation_map["Catalan"] = nation_map["Spanish"]
nation_map["Caymanian"] = nation_map["Caribbean"]
nation_map["Ceylon"] = nation_map["Sri Lankan"]
nation_map["Ceylonese"] = nation_map["Sri Lankan"]
nation_map["Chagossian"] = "Indian Ocean"
nation_map["Chechen"] = nation_map["Russian"]
nation_map["Cherokee"] = nation_map["US"]
nation_map["Chilian"] = nation_map["Chilean"]
nation_map["China"] = nation_map["Chinese"]
nation_map["Chiricahua"] = nation_map["US"]
nation_map["Chuvash"] = nation_map["Russian"]
nation_map["Circassian"] = nation_map["Russian"]
nation_map["Columbian"] = nation_map["Colombian"]
nation_map["Congo"] = nation_map["Congolese"]
nation_map["Congoleze"] = nation_map["Congolese"]
nation_map["Cornish"] = nation_map["British"]
nation_map["Costan Rican"] = nation_map["Costa Rican"]
nation_map["Côte d'Ivoire"] = nation_map["Ivorian"]
nation_map["Crimean"] = nation_map["Russian"]
nation_map["Croat"] = nation_map["Croatian"]
nation_map["Curaçaoan"] = nation_map["Dutch"]
nation_map["Curaçaon"] = nation_map["Dutch"]
nation_map["Dagestani"] = nation_map["Russian"]
nation_map["Dahomey"] = "Africa"
nation_map["Dijiboutian"] = nation_map["Djiboutian"]
nation_map["Dolgan"] = nation_map["Russian"]
nation_map["Dominica"] = nation_map["Caribbean"]
nation_map["England"] = nation_map["British"]
nation_map["Englist"] = nation_map["British"]
nation_map["European"] = "Europe"
nation_map["Falkland Islands"] = "South America"
nation_map["Falkland islands"] = nation_map["Falkland Islands"]
nation_map["Falkland"] = nation_map["Falkland Islands"]
nation_map["Faroese"] = nation_map["Danish"]
nation_map["Filipina"] = nation_map["Filipino"]
nation_map["Filipo"] = nation_map["Filipino"]  # verified entry
nation_map["Fillipina"] = nation_map["Filipino"]
nation_map["Finish"] = nation_map["Finnish"]
nation_map["Flemish"] = nation_map["Belgian"]
nation_map["Franch"] = nation_map["French"]  # verified entry
nation_map["Franco"] = nation_map["French"]
nation_map["Frenck"] = nation_map["French"]  # verified entry
nation_map["Fujianese"] = nation_map["Chinese"]
nation_map[
    "Gaelic"
] = "Europe"  # refers to sport of Gaelic football, otherwise language
nation_map["Galician"] = nation_map["Spanish"]
nation_map["Galápagos"] = nation_map["Ecuadorian"]  # entry for tortoise
nation_map["Geman"] = nation_map["German"]  # verified entry
nation_map["Germen"] = nation_map["German"]  # verified entry
nation_map["Ghanese"] = "Africa"
nation_map["Greenlandic"] = nation_map["Danish"]
nation_map["Guadeloupean"] = nation_map["Caribbean"]
nation_map["Guamanian"] = "Oceania"
nation_map["Guernsey"] = nation_map["British"]
nation_map["Guernseyan"] = nation_map["British"]
nation_map["Hawaiian"] = nation_map["US"]
nation_map["Hindi"] = nation_map["Indian"]
nation_map["Hindu"] = nation_map["Indian"]
nation_map["Hollywood"] = nation_map["US"]
nation_map["Hong Kong"] = nation_map["Chinese"]
nation_map["Houston"] = nation_map["US"]
nation_map["Huaorani"] = nation_map["Ecuadorian"]
nation_map["I Kiribati"] = nation_map["IKiribati"]
nation_map["Indin"] = nation_map["Indian"]  # verified entry
nation_map["Indo"] = nation_map["Indian"]
nation_map["Ingush"] = nation_map["Russian"]
nation_map["Italo"] = nation_map["Italian"]
nation_map["Ivoirian"] = "Africa"
nation_map["Javanese"] = nation_map["Indonesian"]
nation_map["Jersey"] = nation_map["British"]
nation_map["Kabardin"] = nation_map["Russian"]
nation_map["Kashmiri"] = nation_map["Indian"]
nation_map["Korean"] = "Asia"
nation_map["Kosovan"] = "Europe"
nation_map["Kosovar"] = "Europe"
nation_map["Kosovo"] = "Europe"
nation_map["Kurdish"] = "Asia"
nation_map["Lesothan"] = "Africa"
nation_map["Los Angeles"] = nation_map["US"]
nation_map["Louisiana"] = nation_map["US"]
nation_map["MGerman"] = nation_map["German"]  # verified entry
nation_map["Macanese"] = nation_map["Chinese"]
nation_map["Malayalam"] = nation_map["Indian"]
nation_map["Malayali"] = nation_map["Indian"]
nation_map["Malayan"] = nation_map["Malaysian"]
nation_map["Manx"] = nation_map["British"]
nation_map["Mexian"] = nation_map["Mexican"]
nation_map["Mississippi"] = nation_map["US"]
nation_map["Monegasque"] = nation_map["Monacan"]
nation_map["Montserrat"] = nation_map["Caribbean"]
nation_map["Montserratian"] = nation_map["Caribbean"]
nation_map["Myanmar"] = nation_map["Burmese"]
nation_map["New York"] = nation_map["US"]
nation_map["Ngarrindjeri"] = nation_map["Australian"]
nation_map["Ni Vanuatu"] = "Oceania"
nation_map["Nigirean"] = nation_map["Nigerian"]
nation_map["Niuean"] = nation_map["NZ"]
nation_map["Northern Ire"] = nation_map["Northern Irish"]
nation_map["Northern Ireland"] = nation_map["Northern Irish"]
nation_map["Norther Irish"] = nation_map["Northern Irish"]
nation_map["North Irish"] = nation_map["Northern Irish"]
nation_map["North American"] = "North America"
nation_map["North Island"] = nation_map["NZ"]
nation_map["Northern Mariana Island"] = "Oceania"
nation_map["Northern Mariana Islander"] = "Oceania"
nation_map["Nubian"] = nation_map["Sudanese"]
nation_map["Ottoman"] = nation_map["Turkish"]
nation_map["Paraguan"] = nation_map["Paraguayan"]  # verified entry
nation_map["People's Republic of China"] = nation_map["Chinese"]
nation_map["Pitcairn"] = "Oceania"
nation_map["Poliosh"] = nation_map["Polish"]  # verified entry
nation_map["Polis"] = nation_map["Polish"]  # verified entry
nation_map["Prussian"] = nation_map["German"]
nation_map["Punjabi"] = nation_map["Indian"]
nation_map["Quebec"] = nation_map["Canadian"]
nation_map["Québécois"] = nation_map["Canadian"]
nation_map["Republic of China"] = nation_map["Chinese"]
nation_map["Rhodesian"] = "Africa"
nation_map["Roman"] = nation_map["Italian"]
nation_map["Réunionese"] = nation_map["French"]
nation_map["S African"] = nation_map["South African"]
nation_map["Saban"] = nation_map["Caribbean"]
nation_map["Saharawi"] = "Africa"
nation_map["Sahrawi"] = nation_map["Saharawi"]
nation_map["Saint Helena"] = "South Atlantic"
nation_map["Saint Vincent"] = nation_map["Caribbean"]
nation_map["Saint Martin"] = nation_map["Caribbean"]
nation_map["Saint Pierre and Miquelon"] = "North America"
nation_map["Salvadorean"] = nation_map["Salvadoran"]
nation_map["Sanmarinese"] = nation_map["Sammarinese"]
nation_map["Santomean"] = nation_map["São Toméan"]
nation_map["Seychellian"] = nation_map["Seychellois"]
nation_map["Sicilian"] = nation_map["Italian"]
nation_map["Sicillian"] = nation_map["Italian"]
nation_map["Sikkimese"] = nation_map["Indian"]
nation_map["Sorbian"] = nation_map["German"]
nation_map["South Afican"] = nation_map["South African"]
nation_map["South Ossetian"] = nation_map["Georgian"]
nation_map["Soviet"] = "United Socialist Soviet Republic"
nation_map["Sri lankan"] = nation_map["Sri Lankan"]
nation_map["St Lucian"] = nation_map["Caribbean"]
nation_map["St Kitts and Nevis"] = nation_map["Kittian and Nevisian"]
nation_map["Sumatran"] = nation_map["Indonesian"]
nation_map["Swish"] = nation_map["Swiss"]  # verified entry
nation_map["Tahitian"] = "South Pacific"
nation_map["Tamil"] = "Indian subcontinent"
nation_map["Tasmanian"] = nation_map["Australian"]
nation_map["Telugu"] = nation_map["Indian"]
nation_map["Texas"] = nation_map["US"]
nation_map["Florida"] = nation_map["US"]
nation_map["Tibetan"] = nation_map["Chinese"]
nation_map["Tirkish"] = nation_map["Turkish"]
nation_map["Tirkish"] = nation_map["Turkish"]
nation_map["Trinbagonian"] = nation_map["Trinidadian"]
nation_map["Trinidad"] = nation_map["Trinidadian"]
nation_map["Turks"] = nation_map["Turkish"]
nation_map["U S"] = nation_map["US"]
nation_map["UAE's"] = nation_map["Emirati"]
nation_map["United Kingdom"] = nation_map["British"]
nation_map["Upper Silesian"] = nation_map["Polish"]
nation_map["Uruguyan"] = nation_map["Uruguayan"]
nation_map["Uyghur"] = "Asia"
nation_map["Ni Vanuatu"] = nation_map["Vanuatuan"]
nation_map["Wallisian"] = "Oceania"
nation_map["Xhosa"] = nation_map["South African"]
nation_map["Yazidi"] = "Asia"
nation_map["Yellowstone"] = nation_map["US"]
nation_map["Yugoslav"] = nation_map["Serbian"]
nation_map["Yugoslavia"] = nation_map["Serbian"]
nation_map["Yugoslavian"] = nation_map["Serbian"]
nation_map["Zairean"] = nation_map["Congolese"]
nation_map["Zanzibari"] = nation_map["Tanzanian"]

<IPython.core.display.Javascript object>

#### Example of Checking Rows with Unique Value

In [20]:
# Cell for checking rows of unique values -- used repeatedly while hard-coding above
check_value = "Yellowstone "

check_list = []
for index in df[df["info_2"].notna()].index:
    item = df.loc[index, "info_2"]
    if item:
        if item.startswith(check_value):
            check_list.append(index)
df.loc[check_list, :]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
58424,6,O-Six,", 6, Yellowstone National Park gray wolf, shot.",https://en.wikipedia.org/wiki/O-Six,16,2012,December,None,None,Yellowstone National Park gray wolf,shot,None,None,None,None,None,None,None,None,6.0,None,NaN,NaN


<IPython.core.display.Javascript object>

#### Appending Other Species List for New Species Observed During Hard-coding

In [21]:
# Adding more species to other_species
other_species_df = pd.read_csv("other_species.csv")
other_species = other_species_df["species"].tolist()
other_species.append("kiwi")
other_species.append("stallion")
other_species.append("colt")
other_species.append("Thoroughbred hurdler")
other_species.append("race horse")
other_species.append("racing filly")
other_species.append("wolf")

<IPython.core.display.Javascript object>

#### Observations:
- That was more brute force than we would like to be using, but as the nationality descriptions are mixed in with other features, the approach was chosen over using an approximate match approach such as the fuzzywuzzy library.
- We will proceed to re-run our code searching `info_2`.

#### Checking `info_2` for `place_1` with Updated `nation_map`

In [22]:
%%time

# Column to check
column = "info_2"

# Extract to column
extract_to = "place_1"


# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if item.startswith(nationality) or item.startswith(country):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
            )

# Chime notification when cell successfully executes
chime.success()

CPU times: total: 10.5 s
Wall time: 10.5 s


<IPython.core.display.Javascript object>

In [23]:
# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
127437,21,Lou Brooks,", 77, American graphic artist and cartoonist.",https://en.wikipedia.org/wiki/Lou_Brooks,20,2021,November,None,None,graphic artist and cartoonist,None,None,None,None,None,None,None,None,None,77.0,None,United States of America,NaN
81050,19,David Green,", 76, Welsh cricketer .","https://en.wikipedia.org/wiki/David_Green_(cricketer,_born_1939)",3,2016,March,"Lancashire, Gloucestershire",None,cricketer,None,None,None,None,None,None,None,None,None,76.0,None,Wales,NaN


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for `place_1`

In [24]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')

There are 643 remaining missing values for place_1.



<IPython.core.display.Javascript object>

#### Obervations:
- We have narrowed down the missing values for `place_1` values to just under 650.
- Let us take another look at the remaining unique values for `info_2`.

#### Checking Remaining Unique Values for `info_2` Where `place_1` Value is Missing

In [25]:
# Checking remaining unique values for info_2 where place_1 value is missing
df[df["place_1"].isna()]["info_2"].unique()

array(['Royal Netherlands Navy vice admiral', 'President of Laos',
       'Governor general of the Bahamas',
       'Amateur violinist and philanthropist',
       'West German long distance runner and Olympian',
       'Composer and music editor',
       'President of the Yemen Arab Republic',
       'Native American tribal leader', 'Prime Minister of Rwanda',
       'Male Hungarian international table tennis player',
       '37th President of the United States',
       'Queen of Jordan as the wife of King Talal',
       'Prime Minister of Zaire under Mobutu Sese Seko',
       'President of Burma and writer',
       'Founder and first leader of North Korea',
       'Moravian American classical pianist', 'President of Ciskei',
       'Prime Minister of Nepal', 'President of Palau',
       'Poet and an Esperantist professor', 'East German politician',
       'Jewish rabbi', '17th Indian Army officer and Chief of Staff',
       'All American baseball player',
       'pioneering American b

<IPython.core.display.Javascript object>

#### Observations:
- We can see some country names embedded in the middle of the values, so we will search there next, for nationality or country.
- The first value found will be maintained, which does bias the value toward nationalities/countries higher up on the list, as we are not proceeding from left to right for this iteration.

#### Checking `info_2` for `place_1` via Nationality or Country within Value

In [26]:
%%time

# Column to check
column = "info_2"

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if nationality in item or country in item:
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
            )

# Chime notification when cell successfully executes
chime.success()

CPU times: total: 4.7 s
Wall time: 4.7 s


<IPython.core.display.Javascript object>

In [27]:
# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
83312,22,Dave Bald Eagle,", 97, American Lakota Chief and actor.",https://en.wikipedia.org/wiki/Dave_Bald_Eagle,9,2016,July,None,None,Lakota Chief and actor,None,None,None,None,None,None,None,None,None,97.0,None,United States of America,NaN
114116,9,Brendan Halligan,", 84, Irish economist and politician, Senator .",https://en.wikipedia.org/wiki/Brendan_Halligan,20,2020,August,1973 1977 and MEP 1979 1984,None,economist and politician,Senator,None,None,None,None,None,None,None,None,84.0,None,Ireland,NaN


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for `place_1`

In [28]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')

There are 285 remaining missing values for place_1.



<IPython.core.display.Javascript object>

#### Observations:
- We have 285 remaining missing values for `place_1`.
- After checking these rows, we can update the dictionary again.

In [29]:
# Checking remaining unique values for info_2 where place_1 value is missing
df[df["place_1"].isna()]["info_2"].unique()

array(['Governor general of the Bahamas',
       'Amateur violinist and philanthropist',
       'Composer and music editor',
       'Prime Minister of Zaire under Mobutu Sese Seko',
       'President of Ciskei', 'Poet and an Esperantist professor',
       'Jewish rabbi',
       "native leader and historian of the Metepenagiag Mi'kmaq Nation",
       'sidecarcross rider and the first ever Sidecarcross World Championship',
       'Major League Baseball player', 'professional wrestler',
       'Field hockey player', 'fifth dean of the Harvard Business School',
       'Racecar driver', 'longtime chairman of Liverpool F C',
       'Orthodox Jewish rabbi',
       'Royal Air Force Air marshal and flying ace during World War II',
       'Archdeacon of Halifax',
       'inspector general and executive director of the CIA',
       'broadcast journalist for NBC News who was news anchor of',
       'younger son of Ayatollah Ruhollah Khomeini and father of Hassan Khomeini',
       'actor and occult

<IPython.core.display.Javascript object>

#### Hard-coding Additional Variations on Nationality

In [30]:
# Hard-coding remaining unique nationality/location descriptors
nation_map["Bahamas"] = nation_map["Bahamian"]
nation_map["Zaire"] = nation_map["Zairean"]
nation_map["Ciskei"] = "Africa"
nation_map["Metepenagiag Mi'kmaq"] = nation_map["Canadian"]
nation_map["Major League"] = nation_map["US"]
nation_map["Liverpool"] = nation_map["British"]
nation_map["Halifax"] = nation_map["Canadian"]
nation_map["CIA"] = nation_map["US"]
nation_map["Major Leagues"] = nation_map["US"]
nation_map["Levi Strauss & Co"] = nation_map["US"]
nation_map["Miami"] = nation_map["US"]
nation_map["Heaven's Gate"] = nation_map["US"]
nation_map["Royal Navy"] = nation_map["British"]
nation_map["Luftwaffe"] = nation_map["German"]
nation_map["IRA"] = nation_map["Irish"]
nation_map["Ashanti"] = "Africa"
nation_map["White House"] = nation_map["US"]
nation_map["Boeing"] = nation_map["US"]
nation_map["Red Army"] = nation_map["Soviet"]
nation_map["Yokohama"] = nation_map["Japanese"]
nation_map["Tasmania"] = nation_map["Tasmanian"]
nation_map["Harvard"] = nation_map["US"]
nation_map["Malaya"] = nation_map["Malayan"]
nation_map["Pennsylvania"] = nation_map["US"]
nation_map["House of Saud"] = nation_map["Saudi"]
nation_map["Faroe Islands"] = nation_map["Faroese"]
nation_map["Kentucky"] = nation_map["US"]
nation_map["Norfolk"] = nation_map["British"]  # verified entry
nation_map["Rhodesia"] = nation_map["Rhodesian"]
nation_map["Royal Marines"] = nation_map["British"]
nation_map["Rhode Island"] = nation_map["US"]
nation_map["Jerusalem"] = nation_map["Israeli"]
nation_map["Kriegsmarine"] = nation_map["German"]
nation_map["Nazi"] = nation_map["German"]
nation_map["South Carolina"] = nation_map["US"]
nation_map["Rashtriya Swayamsevak Sangh"] = nation_map["Indian"]
nation_map["Ontario"] = nation_map["Canadian"]
nation_map["London"] = nation_map["British"]
nation_map["McDonalds"] = nation_map["US"]
nation_map["Kleiner"] = nation_map["US"]
nation_map["Worldcom"] = nation_map["US"]
nation_map["Reino Aventura"] = nation_map["Mexican"]
nation_map["Palm Beach"] = nation_map["US"]
nation_map["Lawrence Radiation Laboratory"] = nation_map["US"]
nation_map["Royal Air Force"] = nation_map["British"]
nation_map["Britain"] = nation_map["British"]
nation_map["american"] = nation_map["US"]
nation_map["St  Lucian"] = nation_map["St Lucian"]
nation_map["Cook Islands"] = "Oceania"
nation_map["E O  Green Junior High School"] = nation_map["US"]
nation_map["Cook Islander"] = "Oceania"
nation_map["western lowland"] = "Africa"
nation_map["Saharan"] = "Africa"
nation_map["St  Kitts and Nevis"] = nation_map["Kittian"]
nation_map["Jammu and Kashmir"] = "Indian subcontinent"
nation_map["Cook Island"] = "Oceania"
nation_map["New Caledonian"] = "Oceania"

<IPython.core.display.Javascript object>

#### Adding More Values to `other_species`

In [31]:
# Adding more species to other_species
other_species.append("gorilla")
other_species.append("orca")
other_species.append("mammal")
other_species.append("triple crown winner")

<IPython.core.display.Javascript object>

#### Re-checking `info_2` for `place_1` via Nationality or Country within Value after Updating `nation_map`

In [32]:
%%time

# Column to check
column = "info_2"

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if nationality in item or country in item:
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
            )

# Chime notification when cell successfully executes
chime.success()

CPU times: total: 4.62 s
Wall time: 4.62 s


<IPython.core.display.Javascript object>

In [33]:
# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
110512,6,Armando Francioli,", 100, Italian actor .",https://en.wikipedia.org/wiki/Armando_Francioli,3,2020,April,", ,",None,actor,None,None,None,None,None,None,None,None,None,100.0,None,Italy,NaN
92636,6,Johnny Hallyday,", 74, French rock singer and actor, lung cancer.",https://en.wikipedia.org/wiki/Johnny_Hallyday,65,2017,December,"""Requiem pour un fou"", ""Marie"", ""Tous ensemble""",None,rock singer and actor,lung cancer,None,None,None,None,None,None,None,None,74.0,None,France,NaN


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for `place_1`

In [34]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')

There are 201 remaining missing values for place_1.



<IPython.core.display.Javascript object>

#### Observations:
- There are ~200 remaining missing values for `place_1`, which we do not expect to find in `info_2`.
- We can now move on to searching for `place_2` in `info_2`.

#### Checking `info_2` for `place_2`

In [35]:
%%time

# Column to check
column = "info_2"

# Extract to column
extract_to = "place_2"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[
    (df["place_1"].notna()) & (df[extract_to].isna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if item.startswith(nationality) or item.startswith(country):
            df.loc[index, extract_to] = country
            df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
            
# Chime notification when cell successfully executes
chime.success()

CPU times: total: 7min 34s
Wall time: 7min 34s


<IPython.core.display.Javascript object>

In [36]:
# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
85090,31,Klaus Schulten,", 69, German-American physicist.",https://en.wikipedia.org/wiki/Klaus_Schulten,38,2016,October,None,None,American physicist,None,None,None,None,None,None,None,None,None,69.0,None,Germany,United States of America
75093,23,George Horner,", 91, Czech-born American pianist and physician.",https://en.wikipedia.org/wiki/George_Horner_(musician),4,2015,April,None,None,American pianist and physician,None,None,None,None,None,None,None,None,None,91.0,None,Czech Republic,United States of America


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for `place_1` and Number of Entries with Values for `place_2`

In [37]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')
print(f'There are {df["place_2"].notna().sum()} entries with values for place_2.')

There are 201 remaining missing values for place_1.

There are 9204 entries with values for place_2.


<IPython.core.display.Javascript object>

#### Observations:
- We are finished searching `info_2` for place values and can move on to searching the other numbered `info` columns and `info_parenth` for these values.

#### Checking Other `info_` Columns  for Remaining `place_1` Values

In [38]:
%%time

# Columns to search
cols_lst = [
    'info_1',
    'info_3',
    'info_4',
    'info_5',
    'info_6',
    'info_7',
    'info_8',
    'info_9',
    'info_10',
    'info_11',
    'info_parenth'
]

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for column in cols_lst:
    for nationality, country in nation_map.items():
        dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item.startswith(nationality) or item.startswith(country):
                df.loc[index, extract_to] = country
                df.loc[index, column] = (
                    df.loc[index, column].replace(nationality, "").strip().replace(country, "").strip()
                )

# Chime notification when cell successfully executes
chime.success()

CPU times: total: 26.9 s
Wall time: 26.9 s


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for `place_1`

In [39]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')

There are 178 remaining missing values for place_1.



<IPython.core.display.Javascript object>

#### Observations:
- That iteration captured nearly 25 values.
- We will repeat it, but checking for values that start with the country value.

#### Re-checking info_3 through info_11 and info_parenth for place_1 via Nationality or Country within Value

In [40]:
%%time

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for column in cols_lst:
    for nationality, country in nation_map.items():
        dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
        for index in dataframe.index:
            item = df.loc[index, column]
            if nationality in item or country in item:
                df.loc[index, extract_to] = country
                df.loc[index, column] = (
                    df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
                )

# Chime notification when cell successfully executes
chime.success()

CPU times: total: 26 s
Wall time: 26 s


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for `place_1`

In [41]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')

# Checking a sample of rows with missing place_1
df[df["place_1"].isna()].sample(2)

There are 170 remaining missing values for place_1.



,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
61701,18,Max,", 29, beagle, dachshund and terrier mix, world's oldest dog, Tonic–clonic seizure.",https://en.wikipedia.org/wiki/Max_(dog),4,2013,May,None,None,beagle,dachshund and terrier mix,world's oldest dog,Tonic clonic seizure,None,None,None,None,None,None,29.0,None,NaN,NaN
16804,3,Hoyt Curtin,", 78, composer.",https://en.wikipedia.org/wiki/Hoyt_Curtin,5,2000,December,None,None,composer,None,None,None,None,None,None,None,None,None,78.0,None,NaN,NaN


<IPython.core.display.Javascript object>

#### Obervations:
- We have completed the search for `place_1` and the remaining values do appear to be lacking that data.
- The next step is to check these columns for any `place_2` values.

#### Checking `info_3` through `info_11` and `info_parenth` for Remaining `place_2` Values

In [42]:
%%time

# Columns to search
cols_lst = [
    'info_1',
    'info_3',
    'info_4',
    'info_5',
    'info_6',
    'info_7',
    'info_8',
    'info_9',
    'info_10',
    'info_11',
    'info_parenth'
]

# Extract to column
extract_to = "place_2"

# For loop to extract nation data to place column
for column in cols_lst:
    for nationality, country in nation_map.items():
        dataframe = df[(df['place_1'].notna()) & (df[extract_to].isna()) & (df[column].notna())]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item.startswith(nationality) or item.startswith(country):
                df.loc[index, extract_to] = country
                df.loc[index, column] = (
                    df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
                )

# Chime notification when cell successfully executes
chime.success()

CPU times: total: 7min 42s
Wall time: 7min 42s


<IPython.core.display.Javascript object>

#### Observations:
- We should likely stop our extraction of place values here, as searching for embedded `place_2` values is likely to generate erroneous value.
- For example, "English foremost expert on Poland" would result in Poland as a value for `place_2`.  We accepted this possibility for the 30 `place_1` values that were extracted from embedded nationality or country, but we will not do that for `place_2`.
- Let us take a look at a sample of some entries with only `place_1` values, to confirm this decision.

#### Examining Sample of Rows with Only `place_1` Value

In [43]:
%%time

# For loop to check remaining items for place_2 values
rows_to_check = []
for column in cols_lst:
    for nationality, country in nation_map.items():
        dataframe = df[
            (df["place_1"].notna()) & (df["place_2"].isna()) & (df[column].notna())
        ]
        for index in dataframe.index:
            item = df.loc[index, column]
            if nationality in item or country in item:
                rows_to_check.append(index)

# Chime notification when cell successfully executes
chime.success()

CPU times: total: 8min 9s
Wall time: 8min 9s


<IPython.core.display.Javascript object>

In [44]:
df.loc[rows_to_check, :].sample(20)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
96328,2,Álvaro Lapuerta,", 90, Spanish politician and prosecutor, member of Francoist Courts , complications from dementia.",https://en.wikipedia.org/wiki/%C3%81lvaro_Lapuerta,15,2018,June,1967 1977 and Deputy 1977 2004,None,politician and prosecutor,member of Francoist Courts,complications from dementia,None,None,None,None,None,None,None,90.0,None,Spain,NaN
35864,8,Donald C. MacDonald,", 94, Canadian politician, former leader of the Ontario New Democratic Party.",https://en.wikipedia.org/wiki/Donald_C._MacDonald,5,2008,March,None,None,politician,former leader of the Ontario New Democratic Party,None,None,None,None,None,None,None,None,94.0,None,Canada,NaN
110724,10,Tom Webster,", 71, Canadian ice hockey player , brain cancer.",https://en.wikipedia.org/wiki/Tom_Webster_(ice_hockey),10,2020,April,"Detroit Red Wings, New England Whalers and coach Los Angeles Kings",None,ice hockey player,brain cancer,None,None,None,None,None,None,None,None,71.0,None,Canada,NaN
118870,28,Wang Shouguan,", 98, Chinese astronomer, delegate to the National People's Congress and member of the Chinese Academy of Sciences.",https://en.wikipedia.org/wiki/Wang_Shouguan,4,2021,January,1978 1998,None,astronomer,delegate to the National People's Congress and member of the Chinese Academy of Sciences,None,None,None,None,None,None,None,None,98.0,None,"China, People's Republic of",NaN
47212,10,Jim Farry,", 56, Scottish football administrator, Chief Executive of the Scottish Football Association , heart attack.",https://en.wikipedia.org/wiki/Jim_Farry,9,2010,November,1990 1999,None,football administrator,Chief Executive of the Scottish Football Association,heart attack,None,None,None,None,None,None,None,56.0,None,Scotland,NaN
68798,2,Kuaima Riruako,", 79, Namibian chieftain, Paramount Chief of the Herero , hypertension.",https://en.wikipedia.org/wiki/Kuaima_Riruako,7,2014,June,"since 1978, Hereroland Political Representative to South Africa 1978 1980",None,chieftain,Paramount Chief of the Herero,hypertension,None,None,None,None,None,None,None,79.0,None,Namibia,NaN
69500,9,Fred Martinez,", 60, Belizean politician and diplomat, Minister of Trade , Ambassador to Mexico and Guatemala.",https://en.wikipedia.org/wiki/Fred_Martinez,3,2014,July,"1997 1998, Senator 1984 1989, 1997 1998",None,politician and diplomat,Minister of Trade,Ambassador to Mexico and Guatemala,None,None,None,None,None,None,None,60.0,None,Belize,NaN
128944,13,Mario Cámpora,", 91, Argentine diplomat, ambassador to the United Kingdom .",https://en.wikipedia.org/wiki/Mario_C%C3%A1mpora,3,2022,January,1990 1994 and Belgium 1995 1999,None,diplomat,ambassador to the United Kingdom,None,None,None,None,None,None,None,None,91.0,None,Argentina,NaN
130961,20,Wen Shengchang,", 100, Chinese oceanographer and academic , member of the Chinese Academy of Sciences.",https://en.wikipedia.org/wiki/Wen_Shengchang,4,2022,March,Ocean University of China,None,oceanographer and academic,member of the Chinese Academy of Sciences,None,None,None,None,None,None,None,None,100.0,None,"China, People's Republic of",NaN
107417,12,Norman Kingsbury,", 87, New Zealand educational administrator, University of Waikato registrar .",https://en.wikipedia.org/wiki/Norman_Kingsbury,8,2019,December,"1964 1988, New Zealand Qualifications Authority CEO 1990 2000",None,educational administrator,University of Waikato registrar,None,None,None,None,None,None,None,None,87.0,None,New Zealand,NaN


<IPython.core.display.Javascript object>

#### Observations:
- The sample validates the decision to not look for `place_2` values that aren't at the beginning of the column's string value.

#### Final Counts of Missing Values for `place_1` and Number of Entries with Values for `place_2`

In [45]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')
print(f'There are {df["place_2"].notna().sum()} entries with values for place_2.')

There are 170 remaining missing values for place_1.

There are 11409 entries with values for place_2.


<IPython.core.display.Javascript object>

#### Observations:
- We are finished with extracting the place values.
- It's time to save our dataframe and start a new notebook before extracting `known_role` and `cause_of_death` values.
- Exporting `other_species` and `nation_map` is also a good idea, at this time.

## Exporting Dataset to SQLite Database [wp_life_expect_clean3.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_clean3.db)

In [46]:
# Saving complete raw dataset in a SQLite database
conn = sql.connect("wp_life_expect_clean3.db")
df.to_sql("wp_life_expect_clean3", conn, index=False)

132652

<IPython.core.display.Javascript object>

## Saving nation_map to a Pickle File [nation_map.pkl](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/nation_map.pkl)

In [47]:
# Create a binary pickle file
f = open("nation_map.pkl", "wb")

# Write the dictionary to pickle file
pickle.dump(nation_map, f)

# close file
f.close()

<IPython.core.display.Javascript object>

## Exporting other_species List to [other_species.csv](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/other_species.csv)

In [48]:
# Convert other_species list to dataframe and save to csv -- overwriting existing csv file
other_species_df = pd.DataFrame({"species": other_species})
other_species_df.to_csv("other_species.csv", index=False)

<IPython.core.display.Javascript object>

# [Proceed to Notebook 5 of : Data Cleaning Part 4](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean4_thanak_2022_06_23.ipynb)